In [1]:
import pandas as pd
import numpy as np
import gffutils
import pysam

In [ ]:
#This can take a long time, so only run this once (remove the #)

#db = gffutils.create_db('chm13.draft_v2.0.gene_annotation.gff3', dbfn='chm13db.db', force=True, keep_order=True, 
#merge_strategy='create_unique', sort_attribute_values=True) 

#replace gff3 with your gff3 file, dbfn can be whichever name you want to store the transposed file as


In [18]:
db = gffutils.FeatureDB('chm13db.db', keep_order=True)
db

In [19]:
slct = db.region(region=('chr1', 207320867, 207320897), completely_within=False)

In [20]:
records = []
for gene in slct:
    records.append({
        "seqid": gene.seqid,
        "source": gene.source,
        "featuretype": gene.featuretype,
        "start": gene.start,
        "end": gene.end,
        "strand": gene.strand,
        "score": gene.score,
        "attributes": gene.attributes  # Attributes as a dictionary (will be expanded later)
    })

In [ ]:
df = pd.DataFrame(records)
attributes_df = df["attributes"].apply(pd.Series)
df = pd.concat([df.drop(columns=["attributes"]), attributes_df], axis=1)


,seqid,source,featuretype,start,end,strand,score,source_transcript,source_transcript_name,source_gene,...,collapsed_gene_ids,collapsed_gene_names,extra_paralog,exon_anotation_support,novel_5p_cap,novel_poly_a,intron_rna_support,exon_rna_support,rna_support,reference_support
0,chr1,CAT,transcript,207268990,207491104,-,9980,[ENST00000367033.4],[PLXNA2-201],[ENSG00000076356.7],...,[N/A],[N/A],[False],NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,chr1,CAT,gene,207268990,207491209,-,.,[N/A],NaN,[ENSG00000076356.7],...,[N/A],[N/A],[False],[N/A],NaN,NaN,NaN,NaN,NaN,NaN
2,chr1,CAT,transcript,207273418,207491209,-,.,[N/A],NaN,[ENSG00000076356.7],...,[N/A],[N/A],[False],NaN,[True],[True],"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",NaN,NaN
3,chr1,CAT,transcript,207273418,207491209,-,.,[N/A],NaN,[ENSG00000076356.7],...,[N/A],[N/A],[False],NaN,[True],[True],"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",NaN,NaN
4,chr1,CAT,intron,207307614,207326032,-,.,[ENST00000367033.4],[PLXNA2-201],[ENSG00000076356.7],...,[N/A],[N/A],[False],NaN,NaN,NaN,NaN,NaN,[N/A],[True]
5,chr1,CAT,intron,207307614,207326032,-,.,[N/A],NaN,[ENSG00000076356.7],...,[N/A],[N/A],[False],NaN,[True],[True],"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",[True],[True]
6,chr1,CAT,intron,207307614,207326032,-,.,[N/A],NaN,[ENSG00000076356.7],...,[N/A],[N/A],[False],NaN,[True],[True],"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",[True],[True]


In [22]:
for col in df.columns:
    df[col] = df[col].apply(lambda x: x[0] if isinstance(x, list) and len(x) == 1 else x)
df.dropna(how='all', axis=1, inplace=True)
df

,seqid,source,featuretype,start,end,strand,score,source_transcript,source_transcript_name,source_gene,...,collapsed_gene_ids,collapsed_gene_names,extra_paralog,exon_anotation_support,novel_5p_cap,novel_poly_a,intron_rna_support,exon_rna_support,rna_support,reference_support
0,chr1,CAT,transcript,207268990,207491104,-,9980,ENST00000367033.4,PLXNA2-201,ENSG00000076356.7,...,N/A,N/A,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,chr1,CAT,gene,207268990,207491209,-,.,N/A,NaN,ENSG00000076356.7,...,N/A,N/A,False,N/A,NaN,NaN,NaN,NaN,NaN,NaN
2,chr1,CAT,transcript,207273418,207491209,-,.,N/A,NaN,ENSG00000076356.7,...,N/A,N/A,False,NaN,True,True,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",NaN,NaN
3,chr1,CAT,transcript,207273418,207491209,-,.,N/A,NaN,ENSG00000076356.7,...,N/A,N/A,False,NaN,True,True,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",NaN,NaN
4,chr1,CAT,intron,207307614,207326032,-,.,ENST00000367033.4,PLXNA2-201,ENSG00000076356.7,...,N/A,N/A,False,NaN,NaN,NaN,NaN,NaN,N/A,True
5,chr1,CAT,intron,207307614,207326032,-,.,N/A,NaN,ENSG00000076356.7,...,N/A,N/A,False,NaN,True,True,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",True,True
6,chr1,CAT,intron,207307614,207326032,-,.,N/A,NaN,ENSG00000076356.7,...,N/A,N/A,False,NaN,True,True,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",True,True
